In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from VPRM_offline.src.get_modis_point import get_modis_point
from VPRM_offline.src.get_sentinel_point import get_sentinel_point

from VPRM_offline.src.OfflineVPRM import julian
import datetime
from scipy import interpolate
from netCDF4 import Dataset
from os import listdir


cwd = '/data/co2flux/common/mdomhoef/Oslo/Oslo_analysis/'
year = 2017
StationDataPath = cwd + 'data/HH_Data_small/'

stations_file = cwd + 'Stations.csv'
years_file = cwd + 'Station_years.csv'
ERA5path = cwd + 'data/era5/ERA5_'+str(year)+'/'


first_day = datetime.datetime(year,1,1)
last_day = datetime.datetime(year+1,1,1)
num_days = (last_day - first_day).days
time_steps = num_days *48

Outputpath = cwd

satellite_origin = 'SENTINEL2' #Options are MODIS, MODIS_pp and SENTINEL2
# satellite_origin = 'MODIS' #Options are MODIS and SENTINEL2
# satellite_origin = 'MODIS_pp' #Options are MODIS and SENTINEL2

tag = 'SENTINEL2_500m' #tag for simulation output
# tag = 'MODIS_500m' 
# tag = 'MODIS_pp' #tag for simulation output


MODISpath = cwd + 'data/MODIS_OSLO/' #modis preprocessed 
MODISpreprocessed = cwd + 'data/MODIS_OSLO/'

if satellite_origin == 'MODIS':
    MODISpath = cwd + 'data/MODIS/' #modis tiles 
elif satellite_origin == 'SENTINEL2':
    SENTINELpath = cwd + 'data/Sentinel2/'
    res = '_500m'



In [12]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

In [13]:
stations = pd.read_csv(stations_file, sep = ',')
years_st = pd.read_csv(years_file, index_col= 0) 

snames = stations['Station']
vegfra_types = ['Evergreen','Decid', 'Mixfrst','Shrub','Savan','Crop','Grass','Other']
vprm_class = ["Evergreen Forest","Deciduous Forest","Mixed Forest","Shrubland","Savannas","Cropland","Grassland","Others"]

#get the years and stations with data of year 
for station in stations['Station'].unique():
    if years_st.loc[year][station] != 1.0:
        stations = stations.drop(stations[stations['Station'] == station].index)

snames = stations['Station'].unique()
stations.set_index(stations['Station'], inplace=True)
nsites = len(stations)

output_df = pd.DataFrame()
# snames = ['SE-Lnn']
print(snames)
time_steps = num_days *48



['FI-Hyy' 'FI-Let' 'SE-Lnn' 'SE-Ros' 'SE-Deg' 'SE-Htm' 'SE-Nor']


In [14]:
for sitename in snames:
    print('-----------------------------------------------')
    print('Start processing at ' + sitename + ' station')
    # print(stations.columns)

    lon = stations.loc[sitename, 'Longitude']
    lat = stations.loc[sitename, 'Latitude']
    tile = [stations.loc[sitename, 'tile_h'], stations.loc[sitename, 'tile_v']]
    veg_type = stations.loc[sitename, 'VPRM']
    # iveg = igbp_dict[veg_type]
    iveg = veg_type # - 1
    """
    2. ESTIMATION OF HOURLY EVI/LSWI VARIATIONS
    """
    if satellite_origin == 'MODIS_pp':
        print('getting preprocessed MODIS for ', sitename, ' station ', year)
        data = get_modis_point(year=year, lat = lat, lon = lon, tile = tile, MODISpath = MODISpath, sitename=sitename)
    elif satellite_origin == 'MODIS':
        print('getting MODIS for ', sitename, ' station ', year)
        #get LSWI from preprocessed 
        data = get_modis_point(year=year, lat = lat, lon = lon, tile = tile, MODISpath = MODISpreprocessed, sitename=sitename)
        #get EVI from 
        MODISpath_n = MODISpath + str(year) + '_h' + str(tile[0]) + '_v0' + str(tile[1])
        data_1 = get_modis_point_new(year=year, lat = lat, lon = lon, tile = tile, MODISpath = MODISpath_n, sitename=sitename)
        # print(data_1.dtype)
        data[1] = data_1.iloc[1:, 0].values
        
    elif satellite_origin == 'SENTINEL2':
        print('getting Sentinel Data for ', sitename, ' station ', year)
        data = get_sentinel_point(year=year, SENTINELpath= SENTINELpath, res = res, sitename= sitename)
        
    fjul = (julian(1,1, year)-1) + data[0]

    if satellite_origin == 'MODIS':
        fjul_m = fjul[0::2]
    else:
        fjul_m = fjul
    
    # print('--------------------------------------------------')    
    if year%4 ==0:
        fjul_out = (julian(1,1, year)) + np.arange(0,366, step = 1/48)   
    else:
        fjul_out = (julian(1,1, year)) + np.arange(0,365, step = 1/48)
    # data = np.array(data, dtype=np.float32)   
    
    data = [fjul, data[1], data[2]]

    
    if np.isnan(data[1]).all():
        data[1][:] = np.nan
        # data[1][:] = 0.5
        print("all EVI missing for",sitename)
    if np.isnan(data[2]).all():
        data[2][:] = np.nan
        # data[2][:] = 0.5
        print("all LSWI missing for",sitename)
    EVI = np.empty(shape=(time_steps))
    LSWI = np.empty(shape=(time_steps))
    
        
    f = interpolate.interp1d(fjul_m, data[1], fill_value = 'extrapolate')
    EVI[:] = f(fjul_out)
    f = interpolate.interp1d(fjul, data[2], fill_value = 'extrapolate')
    LSWI[:] = f(fjul_out)     
    if sitename == snames[0]:
        sep= np.modf(fjul_out)
        base = datetime.datetime(year = 1960, month = 1, day = 1)
        date_list = [base + datetime.timedelta(days=x)+datetime.timedelta(hours=y) for x, y in zip(sep[1], sep[0]*24)]
        output_df['Times'] = date_list
    
    
    output_df[sitename + '_EVI'] = EVI
    output_df[sitename + '_LSWI'] = LSWI
    
    '''
    3. Getting meterology data 
    '''
    OBS_TA = False
    OBS_SW_IN = False
            
    TEMP = np.array([])
    RAD = np.array([])
    
    for month in range(12):
        met_nc = Dataset(ERA5path+'ERA5_'+str(month+1).zfill(2)+'_' + str(year)+'.nc', 'r')
        if month == 0:
            lat_era5 = np.array(met_nc.variables['latitude'])
            lat_era5 = lat_era5[::-1] #Define latitudes from lower values to higher values
            lon_era5 = np.array(met_nc.variables['longitude'])

            dlat = abs(lat-lat_era5)
            dlon = abs(lon -lon_era5)
                        
            sela = np.where(dlat == np.min(dlat))[0][0]
            selo = np.where(dlon == np.min(dlon))[0][0]  
            # print(lat_era5[sela],lon_era5[selo])
            if lat_era5[sela] >= lat:
                if lon_era5[selo] >= lon:
                    ISW = selo -1
                    JSW = sela -1   
                else:
                    ISW = selo
                    JSW = sela -1
            else:
                if lon_era5[selo] >= lon:
                    ISW = selo -1
                    JSW = sela 
                else:
                    ISW = selo
                    JSW = sela 
            factorNE = ((lat - lat_era5[JSW])/(lat_era5[JSW+1] - lat_era5[JSW]))*((lon - lon_era5[ISW])/(lon_era5[ISW+1] - lon_era5[ISW]))
            factorSE = ((lat_era5[JSW + 1] - lat)/(lat_era5[JSW+1] - lat_era5[JSW]))*((lon - lon_era5[ISW])/(lon_era5[ISW+1] - lon_era5[ISW]))
            factorSW = ((lat_era5[JSW + 1] - lat)/(lat_era5[JSW+1] - lat_era5[JSW]))*((lon_era5[ISW + 1] - lon)/(lon_era5[ISW+1] - lon_era5[ISW]))
            factorNW = ((lat - lat_era5[JSW])/(lat_era5[JSW+1] - lat_era5[JSW]))*((lon_era5[ISW + 1] - lon)/(lon_era5[ISW+1] - lon_era5[ISW]))

        if not OBS_TA:
            temp_era5 = np.array(met_nc.variables['t2m']) - 273.15
            temp_era5 = temp_era5[:,::-1,:]
            temp_out = factorNE*temp_era5[:,JSW + 1, ISW + 1] + factorNW*temp_era5[:,JSW + 1, ISW] + factorSE*temp_era5[:,JSW, ISW + 1] + factorSW*temp_era5[:,JSW, ISW]
            TEMP = np.concatenate((TEMP, temp_out))
        if not OBS_SW_IN:
            ssrd_era5 = np.array(met_nc.variables['ssrd'])
            ssrd_era5 = ssrd_era5[:,::-1,:]/3600
            ssrd_out = factorNE*ssrd_era5[:,JSW + 1, ISW + 1] + factorNW*ssrd_era5[:,JSW + 1, ISW] + factorSE*ssrd_era5[:,JSW, ISW + 1] + factorSW*ssrd_era5[:,JSW, ISW]
            
            RAD = np.concatenate((RAD, ssrd_out))
        met_nc.close()
    if year%4 ==0:
        fjul = (julian(1,1, year)) + np.arange(0,366, step = 1/24)   
    else:
        fjul = (julian(1,1, year)) + np.arange(0,365, step = 1/24)
            
    if not OBS_TA:
        # Temp = interpolate.interp1d(fjul_out, TEMP, fill_value = 'extrapolate')
        f = interpolate.interp1d(fjul, TEMP, fill_value = 'extrapolate')
        Temp = f(fjul_out) #Interpolated to flux time steps
    else:
        Temp = TEMP
    if not OBS_SW_IN:
        # Rad = interpolate.interp1d(fjul_out, RAD, fill_value = 'extrapolate')
        f = interpolate.interp1d(fjul, RAD, fill_value = 'extrapolate')
        Rad = f(fjul_out) #Interpolated to flux time steps
    else:
        Rad = RAD
        
        
    output_df[sitename + '_TEMP_ERA5'] = Temp
    output_df[sitename + '_RAD_ERA5'] = Rad
    
    """
    4. adding station data 
    """
    fls = listdir(StationDataPath)
    fls = [x for x, y in zip(fls, [(sitename in file) for file in fls]) if y == True]
    df_obs = pd.read_table(StationDataPath+fls[0], sep=',')
    df_obs.date = pd.to_datetime(df_obs.date)
    df_obs['year'] = df_obs['date'].dt.year
    
    df_obs = df_obs[df_obs['year']==year]
    
    if 'SW_IN_F' in df_obs.columns:
        df_obs.loc[df_obs['SW_IN_F'] < -9990,'SW_IN_F'] = np.nan
        if df_obs['SW_IN_F'].isna().sum() < 1000:
            RAD = df_obs['SW_IN_F'].values
            OBS_SW_IN = True

    if 'TA_F' in df_obs.columns:
        df_obs.loc[df_obs['TA_F'] < -9990,'TA_F'] = np.nan
        if df_obs['TA_F'].isna().sum() < 1000:
            TEMP = df_obs['TA_F'].values
            OBS_TA = True 

    print(OBS_TA, OBS_SW_IN)

    if OBS_TA:
        nans, x= nan_helper(TEMP)
        TEMP[nans]= np.interp(x(nans), x(~nans), TEMP[~nans])
    if OBS_SW_IN:
        nans, x= nan_helper(RAD)
        RAD[nans]= np.interp(x(nans), x(~nans), RAD[~nans])


    output_df[sitename + '_TEMP_Station'] = TEMP
    output_df[sitename + '_RAD_Station'] = RAD

    


-----------------------------------------------
Start processing at FI-Hyy station
getting Sentinel Data for  FI-Hyy  station  2017
True True
-----------------------------------------------
Start processing at FI-Let station
getting Sentinel Data for  FI-Let  station  2017
True True
-----------------------------------------------
Start processing at SE-Lnn station
getting Sentinel Data for  SE-Lnn  station  2017
True True
-----------------------------------------------
Start processing at SE-Ros station
getting Sentinel Data for  SE-Ros  station  2017
True True
-----------------------------------------------
Start processing at SE-Deg station
getting Sentinel Data for  SE-Deg  station  2017
True True
-----------------------------------------------
Start processing at SE-Htm station
getting Sentinel Data for  SE-Htm  station  2017
True True
-----------------------------------------------
Start processing at SE-Nor station
getting Sentinel Data for  SE-Nor  station  2017
True True


In [15]:
output_df.head(20) 

,Times,FI-Hyy_EVI,FI-Hyy_LSWI,FI-Hyy_TEMP_ERA5,FI-Hyy_RAD_ERA5,FI-Hyy_TEMP_Station,FI-Hyy_RAD_Station,FI-Let_EVI,FI-Let_LSWI,FI-Let_TEMP_ERA5,...,SE-Htm_TEMP_ERA5,SE-Htm_RAD_ERA5,SE-Htm_TEMP_Station,SE-Htm_RAD_Station,SE-Nor_EVI,SE-Nor_LSWI,SE-Nor_TEMP_ERA5,SE-Nor_RAD_ERA5,SE-Nor_TEMP_Station,SE-Nor_RAD_Station
0,2017-01-01 00:00:00,0.319510,0.399635,1.674237,-6.467518e-14,1.962,0.000,0.344234,0.191169,1.584588,...,6.963361,-6.467518e-14,7.278,0.000,0.261788,0.300733,4.432078,-6.467518e-14,5.592,0.000
1,2017-01-01 00:30:00,0.319500,0.399547,1.480078,-6.467518e-14,1.521,0.000,0.344178,0.191181,1.345525,...,6.958217,-6.467518e-14,7.371,0.000,0.261865,0.300704,4.271053,-6.467518e-14,5.588,0.000
2,2017-01-01 01:00:00,0.319490,0.399460,1.285919,-6.467518e-14,1.187,0.000,0.344122,0.191193,1.106462,...,6.953072,-6.467518e-14,7.428,0.000,0.261941,0.300674,4.110027,-6.467518e-14,5.365,0.000
3,2017-01-01 01:30:00,0.319481,0.399372,0.994200,-6.467518e-14,1.127,0.000,0.344066,0.191205,0.800422,...,6.965964,-6.467518e-14,7.467,0.000,0.262018,0.300645,3.985681,-6.467518e-14,5.133,0.000
4,2017-01-01 02:00:00,0.319471,0.399285,0.702481,-6.467518e-14,0.936,0.000,0.344010,0.191218,0.494382,...,6.978855,-6.467518e-14,7.415,0.000,0.262094,0.300616,3.861335,-6.467518e-14,4.883,0.000
5,2017-01-01 02:30:00,0.319461,0.399198,0.426622,-6.467518e-14,0.847,0.000,0.343954,0.191230,0.184164,...,6.997495,-6.467518e-14,7.350,0.000,0.262171,0.300587,3.732376,-6.467518e-14,4.841,0.000
6,2017-01-01 03:00:00,0.319451,0.399110,0.150762,-6.467518e-14,0.786,0.129,0.343899,0.191242,-0.126054,...,7.016135,-6.467518e-14,7.319,0.000,0.262248,0.300558,3.603416,-6.467518e-14,4.740,0.000
7,2017-01-01 03:30:00,0.319441,0.399023,-0.067354,-6.467518e-14,0.399,0.429,0.343843,0.191254,-0.229201,...,6.974845,-6.467518e-14,7.242,0.000,0.262324,0.300528,3.174457,-6.467518e-14,4.555,0.000
8,2017-01-01 04:00:00,0.319432,0.398935,-0.285470,-6.467518e-14,-0.228,0.000,0.343787,0.191266,-0.332348,...,6.933554,-6.467518e-14,7.180,0.000,0.262401,0.300499,2.745498,-6.467518e-14,4.461,0.000
9,2017-01-01 04:30:00,0.319422,0.398848,-0.632293,-6.467518e-14,-0.633,0.000,0.343731,0.191278,-0.621294,...,6.840722,-6.467518e-14,7.102,0.000,0.262477,0.300470,2.057594,-6.467518e-14,4.224,0.000


In [16]:
output_df.to_csv(cwd+'Station_evi_lswi_temp_rad'+'_'+str(year)+'.csv', index = False, header=True)


In [17]:
df_obs.columns

Index(['Unnamed: 0', 'TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F_MDS',
       'TA_F_MDS_QC', 'TA_F', 'TA_F_QC', 'SW_IN_F_MDS', 'SW_IN_F', 'VPD_F_MDS',
       'VPD_F', 'TS_F_MDS_1', 'TS_F_MDS_2', 'TS_F_MDS_3', 'TS_F_MDS_4',
       'TS_F_MDS_5', 'TS_F_MDS_1_QC', 'TS_F_MDS_2_QC', 'TS_F_MDS_3_QC',
       'TS_F_MDS_4_QC', 'TS_F_MDS_5_QC', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'RECO_NT_VUT_REF', 'GPP_NT_VUT_REF', 'RECO_DT_VUT_REF',
       'GPP_DT_VUT_REF', 'date', 'year'],
      dtype='object')

In [18]:
output_df.isnull()

,Times,FI-Hyy_EVI,FI-Hyy_LSWI,FI-Hyy_TEMP_ERA5,FI-Hyy_RAD_ERA5,FI-Hyy_TEMP_Station,FI-Hyy_RAD_Station,FI-Let_EVI,FI-Let_LSWI,FI-Let_TEMP_ERA5,...,SE-Htm_TEMP_ERA5,SE-Htm_RAD_ERA5,SE-Htm_TEMP_Station,SE-Htm_RAD_Station,SE-Nor_EVI,SE-Nor_LSWI,SE-Nor_TEMP_ERA5,SE-Nor_RAD_ERA5,SE-Nor_TEMP_Station,SE-Nor_RAD_Station
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17516,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17517,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17518,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
# df = output_df.filter(regex='FI-Let')

In [20]:
# df